Convolution Layer (ConvLayer): Applies convolutional filters to the input image. It also performs a ReLU activation after each convolution.
Max Pooling Layer (MaxPoolLayer): Reduces the spatial dimensions by keeping only the maximum value in a window.
Fully Connected Layer (FullyConnected): Flattens the output from the previous layer and applies a dense layer to get final outputs.
CNN Class: Combines these layers and provides a forward and backward pass.

In [ ]:
import numpy as np

# Define activation functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def softmax(x):
    exps = np.exp(x - np.max(x))
    return exps / np.sum(exps, axis=0)

# Convolution layer class
class ConvLayer:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        # Initialize filters with random values
        self.filters = np.random.randn(num_filters, filter_size, filter_size) / 9

    def iterate_regions(self, image):
        h, w = image.shape
        for i in range(h - self.filter_size + 1):
            for j in range(w - self.filter_size + 1):
                region = image[i:(i + self.filter_size), j:(j + self.filter_size)]
                yield region, i, j

    def forward(self, input):
        self.input = input
        h, w = input.shape
        output = np.zeros((h - self.filter_size + 1, w - self.filter_size + 1, self.num_filters))
        for region, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(region * self.filters, axis=(1, 2))
        return relu(output)

    def backward(self, d_L_d_out, learning_rate):
        d_L_d_filters = np.zeros(self.filters.shape)
        for region, i, j in self.iterate_regions(self.input):
            for f in range(self.num_filters):
                d_L_d_filters[f] += d_L_d_out[i, j, f] * region

        # Update filters
        self.filters -= learning_rate * d_L_d_filters
        return d_L_d_filters

# Max Pooling layer class
class MaxPoolLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def iterate_regions(self, image):
        h, w, num_filters = image.shape
        new_h = h // self.pool_size
        new_w = w // self.pool_size
        for i in range(new_h):
            for j in range(new_w):
                region = image[(i * self.pool_size):(i * self.pool_size + self.pool_size),
                               (j * self.pool_size):(j * self.pool_size + self.pool_size)]
                yield region, i, j

    def forward(self, input):
        self.input = input
        h, w, num_filters = input.shape
        output = np.zeros((h // self.pool_size, w // self.pool_size, num_filters))
        for region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(region, axis=(0, 1))
        return output

    def backward(self, d_L_d_out):
        d_L_d_input = np.zeros(self.input.shape)
        for region, i, j in self.iterate_regions(self.input):
            h, w, f = region.shape
            amax = np.amax(region, axis=(0, 1))
            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        if region[i2, j2, f2] == amax[f2]:
                            d_L_d_input[i * self.pool_size + i2, j * self.pool_size + j2, f2] = d_L_d_out[i, j, f2]
        return d_L_d_input

# Fully connected layer
class FullyConnected:
    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)

    def forward(self, input):
        self.input = input.flatten()
        return relu(np.dot(self.input, self.weights) + self.biases)

    def backward(self, d_L_d_out, learning_rate):
        d_L_d_input = np.dot(d_L_d_out, self.weights.T).reshape(self.input.shape)
        d_L_d_weights = np.dot(self.input[:, np.newaxis], d_L_d_out[np.newaxis, :])
        d_L_d_biases = d_L_d_out

        # Update weights and biases
        self.weights -= learning_rate * d_L_d_weights
        self.biases -= learning_rate * d_L_d_biases
        return d_L_d_input

# Main CNN class
class CNN:
    def __init__(self):
        self.conv = ConvLayer(8, 3)  # 8 filters of size 3x3
        self.pool = MaxPoolLayer(2)  # Max pooling of size 2x2
        self.fc = FullyConnected(13 * 13 * 8, 10)  # Flattened layer to 10 outputs

    def forward(self, image):
        output = self.conv.forward((image / 255) - 0.5)
        output = self.pool.forward(output)
        output = self.fc.forward(output)
        return softmax(output)

    def train(self, image, label, learning_rate=0.005):
        # Forward pass
        output = self.forward(image)

        # Calculate loss (cross-entropy)
        loss = -np.log(output[label])
        gradient = np.zeros(10)
        gradient[label] = -1 / output[label]

        # Backward pass
        grad_back = self.fc.backward(gradient, learning_rate)
        grad_back = self.pool.backward(grad_back)
        self.conv.backward(grad_back, learning_rate)

        return loss
